<center><a href="https://www.nvidia.cn/training/"><img src="https://dli-lms.s3.amazonaws.com/assets/general/DLI_Header_White.png" width="400" height="186" /></a></center>

<br>

# <font color="#76b900">**Notebook 9:** LangServe 和评估</font>

<br>

## LangServe 服务器设置

这个 notebook 是为那些对使用 LangChain 和 [**LangServe**](https://python.langchain.com/docs/langserve) 开发交互式 Web 应用感兴趣的人提供的一个游乐场。目的是提供一个最小代码示例，展示 LangChain 在 Web 应用场景中的潜力。

本节提供了一个使用 LangChain 的运行时（Runnable）接口与 FastAPI 设置简单 API 服务器的流程示例。这个示例演示了如何集成 LangChain 模型，例如 `ChatNVIDIA`，来创建和分发可访问的 API 路由。通过这个，您将能够为前端服务的 [**`frontend_server.py`**](frontend/frontend_server.py) 会话提供功能，该会话期望：
- 一个名为 `:9012/basic_chat` 的简单入口，用于基本聊天机器人，示例如下。
- 一对名为 `:9012/retriever` 和 `:9012/generator` 的入口，用于 RAG 聊天机器人。
- 全部三个入口用于进行**评估**。*稍后会详细说明！*

**重要提示：**
- 确保点击方框（ $\square$ ）按钮两次以关闭运行中的 FastAPI 单元。第一次点击可能会失败或在异步进程上触发 try-catch。
- 如果仍然无法正常工作，请用 **Kernel -> Restart Kernel** 对这个 notebook 进行强制重启。
- 当 FastAPI 服务器在您的单元中运行时，预计该进程会阻塞这个 notebook。其它 notebook 不应该受到影响。

<br>

### **第 1 部分：** 提供 /basic_chat 入口

提供了作为独立 Python 文件启动 `/basic_chat` 入口的说明。这将被前端用于做出基本决策，而无需内部推理。

In [ ]:
%%writefile server_app.py
# https://python.langchain.com/docs/langserve#server
from fastapi import FastAPI
from langchain_nvidia_ai_endpoints import ChatNVIDIA, NVIDIAEmbeddings
from langserve import add_routes

## May be useful later
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate, PromptTemplate
from langchain_core.prompt_values import ChatPromptValue
from langchain_core.runnables import RunnableLambda, RunnableBranch, RunnablePassthrough
from langchain_core.runnables.passthrough import RunnableAssign
from langchain_community.document_transformers import LongContextReorder
from functools import partial
from operator import itemgetter

from langchain_community.vectorstores import FAISS

## TODO: Make sure to pick your LLM and do your prompt engineering as necessary for the final assessment
embedder = NVIDIAEmbeddings(model="nvidia/nv-embed-v1", truncate="END")
instruct_llm = ChatNVIDIA(model="meta/llama3-8b-instruct")

app = FastAPI(
  title="LangChain Server",
  version="1.0",
  description="A simple api server using Langchain's Runnable interfaces",
)

## PRE-ASSESSMENT: Run as-is and see the basic chain in action

add_routes(
    app,
    instruct_llm,
    path="/basic_chat",
)

## ASSESSMENT TODO: Implement these components as appropriate

add_routes(
    app,
    RunnableLambda(lambda x: "Not Implemented"),
    path="/generator",
)

add_routes(
    app,
    RunnableLambda(lambda x: []),
    path="/retriever",
)

## Might be encountered if this were for a standalone python file...
if __name__ == "__main__":
    import uvicorn
    uvicorn.run(app, host="0.0.0.0", port=9012)

In [ ]:
## Works, but will block the notebook.
!python server_app.py  

## Will technically work, but not recommended in a notebook. 
## You may be surprised at the interesting side effects...
# import os
# os.system("python server_app.py &")

<br>

### **第 2 部分：** 使用服务器：

虽然在 Google Colab 中无法轻松使用（或者说没有很多特殊技巧的话），但上面的脚本会激活一个与 notebook 进程绑定的运行服务器。服务器运行时，请不要尝试使用这个 notebook（除了关闭/重启服务）。

不过在另一个文件中，您应该能够使用以下接口访问 `basic_chat` 入口：

```python
from langserve import RemoteRunnable
from langchain_core.output_parsers import StrOutputParser

llm = RemoteRunnable("http://0.0.0.0:9012/basic_chat/") | StrOutputParser()
for token in llm.stream("Hello World! How is it going?"):
    print(token, end='')
```

**请在另一个文件中试试看，看看它是否能工作！**


<br>

### **第 3 部分：最终评估**

**这个 notebook 将用于完成最终评估！** 在您完成课程后，我们建议克隆这个 notebook，打开一个新标签页并实现 Evaluate 功能，通过实现上面的 `/generator` 和 `/retriever` 入口！要快速链接到前端，可以运行下面的单元：

In [ ]:
%%js
var url = 'http://'+window.location.host+':8090';
element.innerHTML = '<a style="color:#76b900;" target="_blank" href='+url+'><h2>< Link To Gradio Frontend ></h2></a>';

<hr>
<br>

#### **评估提示：** 
请注意，以下功能已经在前端微服务中实现。 
```python
## Necessary Endpoints
chains_dict = {
    'basic' : RemoteRunnable("http://lab:9012/basic_chat/"),
    'retriever' : RemoteRunnable("http://lab:9012/retriever/"),  ## For the final assessment
    'generator' : RemoteRunnable("http://lab:9012/generator/"),  ## For the final assessment
}

basic_chain = chains_dict['basic']

## Retrieval-Augmented Generation Chain

retrieval_chain = (
    {'input' : (lambda x: x)}
    | RunnableAssign(
        {'context' : itemgetter('input') 
        | chains_dict['retriever'] 
        | LongContextReorder().transform_documents
        | docs2str
    })
)

output_chain = RunnableAssign({"output" : chains_dict['generator'] }) | output_puller
rag_chain = retrieval_chain | output_chain
```
**为了符合这个入口的引入策略，确保不要重复工作流的功能，只部署缺失的特性！**